# Sequence-Structure Comparison with Evol

This tutorial will explain how to compare sequence conservation properties with structural mobility obtained from Gaussian network model (GNM) calculations, and how to perform coevolution analysis.

## Setting up environment

In [ ]:
from prody import *
from pylab import *
%matplotlib inline
confProDy(auto_show=False)

## Entropy Calculation

First, we retrieve MSA for protein family PF00074:

In [ ]:
fetchPfamMSA('PF00074')

We parse the MSA file:

In [ ]:
msa = parseMSA('PF00074_full.sth')

In [ ]:
msa.getLabels()

Then, we refine it using [refineMSA( )](http://prody.csb.pitt.edu/manual/reference/sequence/msa.html?highlight=refinemsa#prody.sequence.msa.refineMSA) based on the sequence of RNAS1_BOVIN:

In [ ]:
msa_refined = refineMSA(msa, label='RNAS1_BOVIN', rowocc=0.8, seqid=0.98)

In [ ]:
msa_refined

We calculate [entropy](http://prody.csb.pitt.edu/manual/reference/sequence/analysis.html?highlight=shannonentropy#prody.sequence.analysis.calcShannonEntropy) for refined MSA.

Shannon's entropy measures the degree of uncertainty that exists in a system. In the case of multiple alignments, the Shannon entropy of each protein site can be computed according to: 

$$H(p_1, p_2, \ldots, p_n) = -\sum_{i=1}^n p_i \log_2 p_i $$

where $p_i$ is the frequency of amino acid $i$ in that site. If a column is completely conserved then Shannon entropy is 0. The maximum variability, where each amino acid occurs with frequency 1/20, yields an entropy of 4.32

In [ ]:
entropy = calcShannonEntropy(msa_refined)

In [ ]:
figure(figsize=(15,6))
showShannonEntropy(msa_refined);

## Mobility calculation

Next, we obtain residue fluctuations or mobility for protein member of the above family. We will use chain B of 2W5I.

In [ ]:
fetchPDBviaHTTP('2W5I')
pdb = parsePDB('2W5I', chain='B', subset='calpha')

A summary of information related to the chosen reference sequence can be retrieved from Uniprot and stored in a dictionary:

In [ ]:
queryUniprot('RNAS1_BOVIN')

Now we need to make sure that the PDB sequence is the same as the reference sequence:

In [ ]:
print(msa_refined['RNAS1_BOVIN'])

In [ ]:
print( pdb.getSequence() )

In [ ]:
print( pdb.getResnums() )

In [ ]:
chB_ca = pdb.select('resid 3 to 121')

In [ ]:
print( msa_refined['RNAS1_BOVIN'] )
print( chB_ca.getSequence() )

We perform GNM as follows:

In [ ]:
gnm = GNM('2W5I')
gnm.buildKirchhoff(chB_ca)
gnm.calcModes(n_modes=None)  # calculate all modes

In [ ]:
showSqFlucts(gnm[0])
showSqFlucts(gnm[:]);

## Plotting

In [ ]:
mobility = calcSqFlucts(gnm)

In [ ]:
figure(figsize=(13,6))

# plot entropy as grey bars
indices = range(1,120)
bar(indices, entropy, width=1.2, color='grey', label='entropy')

# rescale mobility
mobility = mobility*(max(entropy)/max(mobility))

# plot mobility as a blue line
plot(indices, mobility, color='b', linewidth=2, label='mobility')

xlabel('residue index')
ylabel('mobility/entropy');
legend();

In [ ]:
scatter(mobility, entropy)
xlabel('mobility')
ylabel('entropy');
# xlim(.5,1.5);

In [ ]:
corrcoef(mobility, entropy)[0,1]

**NB:** Using only one mode decreases the correlation between conservation and mobility:

In [ ]:
corrcoef(calcSqFlucts(gnm[0]), entropy)[0,1]

## Coevolution Calculation

First we compute the mutual information between the columns in the MSA:

In [ ]:
mutinfo = buildMutinfoMatrix(msa_refined)

In [ ]:
# figure(figsize=(8,8))
showMutinfoMatrix(msa_refined);

However, for **contact prediction** we need a more sophisticated analysis, like the Direct Information (DI):

In [ ]:
coevol = buildDirectInfoMatrix(msa_refined)

In [ ]:
figure(figsize=(8,8))
showDirectInfoMatrix(msa_refined, cmap='inferno');

In [ ]:
showContactMap(gnm, origin='lower', cmap='Greys');

Rank-ordering the DI matrix entries helps to identify the strongest signals:

In [ ]:
rank_row, rank_col, zscore_sort = calcRankorder(coevol, zscore=True)
print( 'row:   ', rank_row[:3] )
print( 'column:', rank_col[:3] )

In [ ]:
showCrossCorr(gnm);

## Visualization on 3D structure

In [ ]:
prot_sel = parsePDB('2W5I', chain='B').select('resid 3 to 121')
resnums = prot_sel.getResnums()
resnums

In order to plot conservation (entropy) values on the full-atom structure, we need to expand the entropy array to assign the same entropy value to all atoms belonging to the same residue:

In [ ]:
entropy_prot = [entropy[r-3] for r in resnums]
entropy_prot

In [ ]:
writePDB('2W5I_entropy.pdb', prot_sel, beta=entropy_prot)

In [ ]:
! ls -1tr

In [ ]:
! vmd 2W5I_entropy.pdb